<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/11_OpenLandMap_SOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract soil measures from OpenLandMap platform

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/carbon_rs_aoi")
# aoi_name = "carbon_rs_aoi"

# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
# aoi_name = "MKR_PACE"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"

# GEE layer ID
layer_dict = {
  "b0": "RS_032",
  "b10": "RS_033",
  "b30": "RS_034",
  "b60": "RS_035"
}


## Sand content processing



In [ ]:
sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02") \
  .select('b0', 'b10', 'b30', 'b60') \
  .clip(aoi)

print(sand.getInfo())
band_list = sand.bandNames().getInfo()
print(type(band_list))
print(band_list)


## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
for i in band_list:
  print(i)
  print(layer_dict[i])

In [ ]:
for i in band_list:
  output_img =  sand.select(i)
  output_name = f"{layer_dict[i]}_{aoi_name}"
  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", i)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:len(band_list)]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

In [ ]:
!pip install geemap --quiet

In [ ]:
import geemap

In [ ]:
vis_params = {"min": 0.0, "max": 100,
                "palette": ['FFFF00','F8F806','F1F10C','EBEB13','E4E419','DDDD20',
    'D7D726','D0D02D','CACA33','C3C33A','BCBC41','B6B647',
    'B0B04E','A9A954','A3A35A','9C9C61','959568','8F8F6E',
    '898975','82827B','7B7B82','757589','6E6E8F','686895',
    '61619C','5A5AA3','5454A9','4D4DB0','4747B6','4141BC',
    '3A3AC3','3333CA','2D2DD0','2626D7','2020DD','1919E4',
    '1212EB','0C0CF1','0606F8','0000FF']
                  }


Map = geemap.Map()
Map.addLayer(sand.select('b0'), vis_params, 'NDVI')

# Map.addLayer(ndvi2011, ndvi_vis_params, 'NDVI-2011')
# Map.addLayer(ndvi2014, ndvi_vis_params, 'NDVI-2014')

Map.centerObject(aoi, zoom=10)
Map